In [1]:
from pynse import *
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter
#import mplfinance as mpf
import time, asyncio
import matplotlib.dates as mdates
import mplfinance as mpf
import statsmodels.api as sm
import statsmodels.formula.api as smf
from datetime import datetime
import os
import glob
import pandas as pd
import mplfinance as mpf
from matplotlib.dates import MonthLocator, DateFormatter
import scipy.stats as stats
from tvDatafeed import TvDatafeed,Interval
tv=TvDatafeed()
import concurrent.futures
import concurrent.futures

nse=Nse()
#nse.bhavcopy(series="all")
#bhavcopy_full=nse.bhavcopy(series="all")

you are using nologin method, data you access may be limited


LOAD DELIV DATA

In [2]:
#########LOADING INDEX########
file_path = r"C:\Users\99111\OneDrive\Desktop\Stocklist.xlsx"
Index = pd.read_excel(file_path)
Index = Index['All'].unique().tolist()

#Price and symbol data
folder_path = "C:/Users/99111/OneDrive/Documents/pynse/Shashank work/data/raw/Delivdata/*.csv"

files = glob.glob(folder_path)

df_list = []

for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

df = pd.concat(df_list)

df = df.rename(columns={'DATE1': 'Date'})
#df = df['SYMBOL'].isin(Index)

df['Open'] = np.where(df['OPEN_PRICE'] > 0, df['OPEN_PRICE'], df['OPEN'])
df['High'] = np.where(df['HIGH_PRICE'] > 0, df['HIGH_PRICE'], df['HIGH'])
df['Low'] = np.where(df['LOW_PRICE'] > 0, df['LOW_PRICE'], df['LOW'])
df['Close'] = np.where(df['CLOSE_PRICE'] > 0, df['CLOSE_PRICE'], df['CLOSE'])
df['Prevclose'] = np.where(df['PREV_CLOSE'] > 0, df['PREV_CLOSE'], df['PREVCLOSE']) 

df['TradedQuantity'] = np.where(df['TTL_TRD_QNTY'] > 0, df['TTL_TRD_QNTY'], df['TOTTRDQTY'])
df['Numberoftrades'] = np.where(df['NO_OF_TRADES'] > 0, df['NO_OF_TRADES'], df['TOTALTRADES'])
df['Averageprice'] = np.where(df['AVG_PRICE'] > 0, df['AVG_PRICE'], (df['Open']+df['High']+df['Low']+df['Close']+df['Prevclose'])/5)

df['TURNOVER_LACS'] = df['TURNOVER_LACS'] / 100
df['TOTTRDVAL'] = df['TOTTRDVAL'] / 10000000
df['Turnover_CR'] = np.where(df['TURNOVER_LACS'] > 0, df['TURNOVER_LACS'], df['TOTTRDVAL'])

df = df.loc[:, ['SYMBOL','Date', 'Prevclose', 'Open', 'High', 'Low', 'Close','Averageprice', 'TradedQuantity', 'Numberoftrades', 'Turnover_CR', 'DELIV_QTY', 'DELIV_PER']]
df['uniquecode'] = df['SYMBOL'] + df['Date'].astype(str)
df.drop_duplicates(subset='uniquecode', keep='first', inplace=True)
df = df.sort_values(by='Date', ascending=True)
df1 = df[df['SYMBOL'].isin(Index)]

##################LOADING SECTORS AND MCAP########################

#Data for PE and MCAP
folder_path = "C:/Users/99111/OneDrive/Desktop/Full data set/Price, PE, EBITDA , EV & Mcap/*.csv"

files = glob.glob(folder_path)

df_list = []

for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

PEMCAP = pd.concat(df_list)
PEMCAP = PEMCAP[PEMCAP['CD_NSE Symbol'].isin(Index)]

PEMCAP['NDP_Date'] = pd.to_datetime(PEMCAP['NDP_Date'])
latestdate = PEMCAP['NDP_Date'].max()
PEMCAP = PEMCAP[PEMCAP['NDP_Date'] >= latestdate]
PEMCAP = PEMCAP.loc[:, ['CD_NSE Symbol', 'CD_Sector' , 'CD_Industry']].copy()
PEMCAP = PEMCAP.rename(columns={'CD_NSE Symbol': 'SYMBOL'})

#Combining data
df2 = pd.merge(df1, PEMCAP, on='SYMBOL', how='left')
df2['Date'] = pd.to_datetime(df2['Date'])

dates = df2['Date'].unique()
dates = dates[-400:]

Testing tool 

In [ ]:
Industry = pd.DataFrame()
Sectors = pd.DataFrame()
Market = pd.DataFrame()

up_sym = pd.DataFrame()
down_sym = pd.DataFrame()


for date in dates[:5]:
    try:

        df3 = df2[df2['Date'] == date]

        df4 = df2[df2['Date'] < date]
        latestdate = df4['Date'].max()
        df4 = df4[df4['Date'] == latestdate]

        df5 = pd.merge(df3, df4, on='SYMBOL', how='inner')
        df5['Change'] = ((((df5['Close_x'] - df5['Prevclose_x']) / df5['Prevclose_x']))*100).round(2)
        df5['up'] = np.where(df5['Change'] > 0, 1, 0)
        df5['down'] = np.where(df5['Change'] < 0, 1, 0)
        df5['Up_Percent'] = np.where(df5['up'] == 1, df5['Change'], 0)
        df5['Down_Percent'] = np.where(df5['down'] == 1, df5['Change'], 0)
        df5['count'] = 1

        df6 = df5.groupby('CD_Industry_x')[['Turnover_CR_x','Prevclose_x', 'Close_x','count', 'Up_Percent', 'up', 'Down_Percent', 'down']].sum().reset_index().copy()
        df6['Up_Change'] = (((df6['Up_Percent'] / df6['up']))*100).round(2)
        df6['Down_Change'] = (((df6['Down_Percent'] / df6['down']))*100).round(2)
        df6 = df6.fillna(0)
        df6['change'] = ((df6['Up_Change'] + df6['Down_Change'])).round(2)
        df6['Sum_Change'] = ((((df6['Close_x'] - df6['Prevclose_x']) / df6['Prevclose_x']))*10000).round(2)
        MTO = df6['Turnover_CR_x'].sum()
        df6['PercentTurnover'] = ((df6['Turnover_CR_x'] / MTO) * 100).round(2)
        df6['Date'] = date
        df6 = df6.loc[:, ['Date','CD_Industry_x', 'Turnover_CR_x', 'PercentTurnover','up', 'down', 'count', 'Up_Change', 'Down_Change','change','Sum_Change']]

        df7 = df5.groupby('CD_Sector_x')[['Turnover_CR_x','Prevclose_x', 'Close_x','count', 'Up_Percent', 'up', 'Down_Percent', 'down']].sum().reset_index().copy()
        df7['Up_Change'] = (((df7['Up_Percent'] / df7['up']))*100).round(2)
        df7['Down_Change'] = (((df7['Down_Percent'] / df7['down']))*100).round(2)
        df7 = df7.fillna(0)
        df7['change'] = ((df7['Up_Change'] + df7['Down_Change'])).round(2)
        df7['Sum_Change'] = ((((df7['Close_x'] - df7['Prevclose_x']) / df7['Prevclose_x']))*10000).round(2)
        MTO = df7['Turnover_CR_x'].sum()
        df7['PercentTurnover'] = ((df7['Turnover_CR_x'] / MTO) * 100).round(2)
        df7['Date'] = date
        df7 = df7.loc[:, ['Date','CD_Sector_x', 'Turnover_CR_x', 'PercentTurnover','up', 'down', 'count', 'Up_Change', 'Down_Change','change','Sum_Change']]
        
        df8 = df5.groupby('Date_x')[['Turnover_CR_x','Prevclose_x', 'Close_x','count', 'Up_Percent', 'up', 'Down_Percent', 'down']].sum().reset_index().copy()
        df8['Up_Change'] = (((df8['Up_Percent'] / df8['up']))*100).round(2)
        df8['Down_Change'] = (((df8['Down_Percent'] / df8['down']))*100).round(2)
        df8 = df8.fillna(0)
        df8['change'] = ((df8['Up_Change'] + df8['Down_Change'])).round(2)
        df8['Sum_Change'] = ((((df8['Close_x'] - df8['Prevclose_x']) / df8['Prevclose_x']))*10000).round(2)

        ups = df5[df5['up'] == 1]
        ups = ups['SYMBOL'].unique().tolist()
        ups = pd.DataFrame({str(date)[:10]: ups})

        downs = df5[df5['down'] == 1]
        downs = downs['SYMBOL'].unique().tolist()
        downs = pd.DataFrame({str(date)[:10]: downs})

        Market = pd.concat([Market, df8], axis=0, ignore_index=True)
        Industry = pd.concat([Industry, df6], axis=0, ignore_index=True)
        Sectors = pd.concat([Sectors, df7], axis=0, ignore_index=True)

        up_sym = pd.concat([up_sym, ups], axis=1, ignore_index=False)
        down_sym = pd.concat([down_sym, downs], axis=1, ignore_index=False)
  

    except Exception as e:
            print(f"Error occurred while creating chart for {date}: {str(e)}")
            continue  


Functional Code

In [3]:
import concurrent.futures

Industry = pd.DataFrame()
Sectors = pd.DataFrame()
Market = pd.DataFrame()

Market_up_sym = pd.DataFrame()
Market_down_sym = pd.DataFrame()

Market_up_p = pd.DataFrame()
Market_down_p = pd.DataFrame()

def process_date(date):
    try:
        df3 = df2[df2['Date'] == date]

        df4 = df2[df2['Date'] < date]
        latestdate = df4['Date'].max()
        df4 = df4[df4['Date'] == latestdate]

        df5 = pd.merge(df3, df4, on='SYMBOL', how='inner')
        df5['Change'] = ((((df5['Close_x'] - df5['Prevclose_x']) / df5['Prevclose_x']))*100).round(2)
        df5['up'] = np.where(df5['Change'] > 0, 1, 0)
        df5['down'] = np.where(df5['Change'] < 0, 1, 0)
        df5['Up_Percent'] = np.where(df5['up'] == 1, df5['Change'], 0)
        df5['Down_Percent'] = np.where(df5['down'] == 1, df5['Change'], 0)
        df5['count'] = 1

        df6 = df5.groupby('CD_Industry_x')[['Turnover_CR_x','Prevclose_x', 'Close_x','count', 'Up_Percent', 'up', 'Down_Percent', 'down']].sum().reset_index().copy()
        df6['Up_Change'] = (((df6['Up_Percent'] / df6['up']))*100).round(2)
        df6['Down_Change'] = (((df6['Down_Percent'] / df6['down']))*100).round(2)
        df6 = df6.fillna(0)
        df6['change'] = ((df6['Up_Change'] + df6['Down_Change'])).round(2)
        df6['Sum_Change'] = ((((df6['Close_x'] - df6['Prevclose_x']) / df6['Prevclose_x']))*10000).round(2)
        MTO = df6['Turnover_CR_x'].sum()
        df6['PercentTurnover'] = ((df6['Turnover_CR_x'] / MTO) * 100).round(2)
        df6['Date'] = date
        df6 = df6.loc[:, ['Date','CD_Industry_x', 'Turnover_CR_x', 'PercentTurnover','up', 'down', 'count', 'Up_Change', 'Down_Change','change','Sum_Change']]

        df7 = df5.groupby('CD_Sector_x')[['Turnover_CR_x','Prevclose_x', 'Close_x','count', 'Up_Percent', 'up', 'Down_Percent', 'down']].sum().reset_index().copy()
        df7['Up_Change'] = (((df7['Up_Percent'] / df7['up']))*100).round(2)
        df7['Down_Change'] = (((df7['Down_Percent'] / df7['down']))*100).round(2)
        df7 = df7.fillna(0)
        df7['change'] = ((df7['Up_Change'] + df7['Down_Change'])).round(2)
        df7['Sum_Change'] = ((((df7['Close_x'] - df7['Prevclose_x']) / df7['Prevclose_x']))*10000).round(2)
        MTO = df7['Turnover_CR_x'].sum()
        df7['PercentTurnover'] = ((df7['Turnover_CR_x'] / MTO) * 100).round(2)
        df7['Date'] = date
        df7 = df7.loc[:, ['Date','CD_Sector_x', 'Turnover_CR_x', 'PercentTurnover','up', 'down', 'count', 'Up_Change', 'Down_Change','change','Sum_Change']]
        
        df8 = df5.groupby('Date_x')[['Turnover_CR_x','Prevclose_x', 'Close_x','count', 'Up_Percent', 'up', 'Down_Percent', 'down']].sum().reset_index().copy()
        df8['Up_Change'] = (((df8['Up_Percent'] / df8['up']))*100).round(2)
        df8['Down_Change'] = (((df8['Down_Percent'] / df8['down']))*100).round(2)
        df8 = df8.fillna(0)
        df8['change'] = ((df8['Up_Change'] + df8['Down_Change'])).round(2)
        df8['Sum_Change'] = ((((df8['Close_x'] - df8['Prevclose_x']) / df8['Prevclose_x']))*10000).round(2)

        ups = df5[df5['up'] == 1]
        ups = ups['SYMBOL'].tolist()
        ups = pd.DataFrame({str(date)[:10]: ups})

        downs = df5[df5['down'] == 1]
        downs = downs['SYMBOL'].tolist()
        downs = pd.DataFrame({str(date)[:10]: downs})

        #ups_p = df5[df5['up'] == 1]
        #ups_p = ups_p['Change'].tolist()
        #ups_p = pd.DataFrame({str(date)[:10]: ups_p})

        #downs_p = df5[df5['down'] == 1]
        #downs_p = downs_p['Change'].tolist()
        #downs_p = pd.DataFrame({str(date)[:10]: downs_p})

        return df6, df7, df8, ups , downs #, ups_p, downs_p

    except Exception as e:
        print(f"Error occurred while processing date {date}: {str(e)}")
        return None, None, None

# Assuming 'dates' is a list of dates
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_date, dates))

# Concatenate the non-empty results to the main dataframes
for result in results:
    if result[0] is not None:
        Industry = pd.concat([Industry, result[0]], axis=0, ignore_index=True)
    if result[1] is not None:
        Sectors = pd.concat([Sectors, result[1]], axis=0, ignore_index=True)
    if result[2] is not None:
        Market = pd.concat([Market, result[2]], axis=0, ignore_index=True)
    if result[3] is not None:
        Market_up_sym = pd.concat([Market_up_sym, result[3]], axis=1, ignore_index=False)
    if result[4] is not None:
        Market_down_sym = pd.concat([Market_down_sym, result[4]], axis=1, ignore_index=False)
    #if result[5] is not None:
    #    Market_up_p = pd.concat([Market_up_p, result[5]], axis=1, ignore_index=False)
    #if result[6] is not None:
    #    Market_down_p = pd.concat([Market_down_p, result[6]], axis=1, ignore_index=False)

Market_up_sym = Market_up_sym.transpose()
Market_up_sym = Market_up_sym.reset_index(drop=False)
Market_up_sym['index'] = pd.to_datetime(Market_up_sym['index'])

Market_down_sym = Market_down_sym.transpose()
Market_down_sym = Market_down_sym.reset_index(drop=False)
Market_down_sym['index'] = pd.to_datetime(Market_down_sym['index'])

#Market_up_p = Market_up_p.transpose()
#Market_up_p = Market_up_p.reset_index(drop=False)
#Market_up_p['index'] = pd.to_datetime(Market_up_p['index'])

#Market_down_p = Market_down_p.transpose()
#Market_down_p = Market_down_p.reset_index(drop=False)
#Market_down_p['index'] = pd.to_datetime(Market_down_p['index'])



In [10]:
Sectors['sectorcodes'] = Sectors['CD_Sector_x'] + Sectors['Date'].astype(str)
Industry['industrycodes'] = Industry['CD_Industry_x'] + Industry['Date'].astype(str)

Secs = Sectors['CD_Sector_x'].unique().tolist()
Inds = Industry['CD_Industry_x'].unique().tolist()

ds = pd.DataFrame()  # Assuming ds is a DataFrame initialized before the loop
di = pd.DataFrame()  # Assuming ds is a DataFrame initialized before the loop

###########################################################

def process_sector(Sec):
    try:
        s = Sectors[Sectors['CD_Sector_x'] == Sec].copy()

        s['TO_Flag'] = (s['Turnover_CR_x'] / s['Turnover_CR_x'].rolling(252).quantile(0.85)).round(2)
        s['PTO_Flag'] = (s['PercentTurnover'] / s['PercentTurnover'].rolling(252).quantile(0.85)).round(2)

        return s
    except Exception as e:
        print(f"Error occurred while processing sector {Sec}: {str(e)}")
        return None

# Assuming Secs is a list of sectors
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_sector, Secs))

# Concatenate the non-empty results to the main DataFrame
ds = pd.concat([result for result in results if result is not None], axis=0, ignore_index=True)

##########################################################

def process_sector(ind):
    try:
        s = Industry[Industry['CD_Industry_x'] == ind].copy()

        s['TO_Flag'] = (s['Turnover_CR_x'] / s['Turnover_CR_x'].rolling(252).quantile(0.85)).round(2)
        s['PTO_Flag'] = (s['PercentTurnover'] / s['PercentTurnover'].rolling(252).quantile(0.85)).round(2)

        return s
    except Exception as e:
        print(f"Error occurred while processing sector {ind}: {str(e)}")
        return 

# Assuming Inds is a list of sectors
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_sector, Inds))

# Concatenate the non-empty results to the main DataFrame
di = pd.concat([result for result in results if result is not None], axis=0, ignore_index=True)


####### Creating prices df###########

symdates = df2['Date'].unique()
symdates = symdates[-15:]
symdates.min()

df9 = df2[df2['Date'] >= symdates.min()].copy()

df9['sector_codes'] = df9['CD_Sector'] + df9['Date'].astype(str)
df9['industry_codes'] = df9['CD_Industry'] + df9['Date'].astype(str)
df9['Change'] = ((((df9['Close'] - df9['Prevclose']) / df9['Prevclose']))*100).round(2)
df10 = df9.loc[:, ['Date','SYMBOL', 'Change', 'uniquecode','CD_Sector','CD_Industry','sector_codes', 'industry_codes']]

##########################################################

Market = Market.sort_values(by='Date_x', ascending=False)
di = di.sort_values(by=['Date', 'Sum_Change'], ascending=[False, False])
ds = ds.sort_values(by=['Date', 'Sum_Change'], ascending=[False, False])
df10 = df10.sort_values(by=['Date', 'Change'], ascending=[False, False])

##########################################################

file_path = r'C:\Users\99111\OneDrive\Desktop\daily updates\Market Overview\Market_Industry_Sectors.xlsx'
Market['Breath'] = (((Market['up'] / Market['count']))*100).round(2)

with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    # Write each DataFrame to a specific sheet
    Market.to_excel(writer, sheet_name='Market', index=False)
    di.to_excel(writer, sheet_name='Industry', index=False)
    ds.to_excel(writer, sheet_name='Sectors', index=False)
    df10.to_excel(writer, sheet_name='price change', index=False)

############# INDUSTRY CODES and SECTOR CODES #################

In [ ]:
symdates = df2['Date'].unique()
symdates = symdates[-5:]
symdates.min()

df9 = df2[df2['Date'] >= symdates.min()].copy()

df9['sector_codes'] = df9['CD_Sector'] + df9['Date'].astype(str)
df9['industry_codes'] = df9['CD_Industry'] + df9['Date'].astype(str)

sector_codes = df9['sector_codes'].unique().tolist()
industry_codes = df9['industry_codes'].unique().tolist()

#########################################################

Ind_up_sym = pd.DataFrame()
Ind_down_sym = pd.DataFrame()

def process_date(industry_code):
    try:
        df10 = df9[df9['industry_codes'] == industry_code].copy()
        df10['Change'] = ((((df10['Close'] - df10['Prevclose']) / df10['Prevclose']))*100).round(2)
        df10['up'] = np.where(df10['Change'] > 0, 1, 0)
        df10['down'] = np.where(df10['Change'] < 0, 1, 0)

        ups = df10[df10['up'] == 1]
        ups1 = ups['SYMBOL'].tolist()
        #ups2 = ups['Change'].tolist()
        ups = pd.DataFrame({industry_code: ups1 })

        downs = df10[df10['down'] == 1]
        downs1 = downs['SYMBOL'].tolist()
        #downs2 = downs['Change'].tolist()
        downs = pd.DataFrame({industry_code: downs1 })


        return  ups , downs

    except Exception as e:
        print(f"Error occurred while processing date {industry_code}: {str(e)}")
        return None, None, None

# Assuming 'dates' is a list of dates
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_date, industry_codes))

# Concatenate the non-empty results to the main dataframes
for result in results:
    if result[0] is not None:
        Ind_up_sym = pd.concat([Ind_up_sym, result[0]], axis=1, ignore_index=False)
    if result[1] is not None:
        Ind_down_sym = pd.concat([Ind_down_sym, result[1]], axis=1, ignore_index=False)

Ind_up_sym = Ind_up_sym.transpose()
Ind_up_sym = Ind_up_sym.reset_index(drop=False)
Ind_up_sym = Ind_up_sym.fillna(0)

Ind_down_sym = Ind_down_sym.transpose()
Ind_down_sym = Ind_down_sym.reset_index(drop=False)
Ind_down_sym = Ind_down_sym.fillna(0)

########################SECTORS####################################################

Sec_up_sym = pd.DataFrame()
Sec_down_sym = pd.DataFrame()

def process_date(sector_code):
    try:
        df10 = df9[df9['sector_codes'] == sector_code].copy()
        df10['Change'] = ((((df10['Close'] - df10['Prevclose']) / df10['Prevclose']))*100).round(2)
        df10['up'] = np.where(df10['Change'] > 0, 1, 0)
        df10['down'] = np.where(df10['Change'] < 0, 1, 0)

        ups = df10[df10['up'] == 1]
        ups1 = ups['SYMBOL'].tolist()
        #ups2 = ups['Change'].tolist()
        ups = pd.DataFrame({sector_code: ups1 })

        downs = df10[df10['down'] == 1]
        downs1 = downs['SYMBOL'].tolist()
        #downs2 = downs['Change'].tolist()
        downs = pd.DataFrame({sector_code: downs1 })


        return  ups , downs

    except Exception as e:
        print(f"Error occurred while processing date {sector_code}: {str(e)}")
        return None, None, None

# Assuming 'dates' is a list of dates
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_date, sector_codes))

# Concatenate the non-empty results to the main dataframes
for result in results:
    if result[0] is not None:
        Sec_up_sym = pd.concat([Sec_up_sym, result[0]], axis=1, ignore_index=False)
    if result[1] is not None:
        Sec_down_sym = pd.concat([Sec_down_sym, result[1]], axis=1, ignore_index=False)

Sec_up_sym = Sec_up_sym.transpose()
Sec_up_sym = Sec_up_sym.reset_index(drop=False)
Sec_up_sym = Sec_up_sym.fillna(0)

Sec_down_sym = Sec_down_sym.transpose()
Sec_down_sym = Sec_down_sym.reset_index(drop=False)
Sec_down_sym = Sec_down_sym.fillna(0)

####### Creating prices df###########
df9['Change'] = ((((df9['Close'] - df9['Prevclose']) / df9['Prevclose']))*100).round(2)
df10 = df9.loc[:, ['Date','SYMBOL', 'Change', 'uniquecode','sector_codes', 'industry_codes']]
df10

############### SAVING FILES ################

In [38]:
with pd.ExcelWriter(r'C:\Users\99111\OneDrive\Desktop\daily updates\Market Overview\Symbols.xlsx', engine='xlsxwriter') as writer:
    Ind_up_sym.to_excel(writer, sheet_name='Ind_up_sym', index=False, header=True)
    Ind_down_sym.to_excel(writer, sheet_name='Ind_down_sym', index=False, header=True)
    
    Sec_up_sym.to_excel(writer, sheet_name='Sec_up_sym', index=False, header=True)
    Sec_down_sym.to_excel(writer, sheet_name='Sec_down_sym', index=False, header=True)

    Market_up_sym.to_excel(writer, sheet_name='Market_up_sym', index=False, header=True)
    Market_down_sym.to_excel(writer, sheet_name='Market_down_sym', index=False, header=True)

    Market_up_p.to_excel(writer, sheet_name='Market_up_sym', index=False, header=True)
    Market_down_p.to_excel(writer, sheet_name='Market_down_sym', index=False, header=True)

    df10.to_excel(writer, sheet_name='pricechange', index=False, header=True)